In [13]:
from data_util import load_embeddings, DMConfig, tfConfig
try:
    import _pickle as cPickle
except ImportError:
    import cPickle
import gzip
import os
import numpy as np
from sklearn import metrics
from defs import LBLS

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
config = DMConfig()
if os.path.exists(config.tok2id_path):
    with gzip.open(os.path.abspath(config.tok2id_path)) as f:
        tok2id = cPickle.load(f)
if os.path.exists(config.id2tok_path):
    with gzip.open(os.path.abspath(config.id2tok_path)) as f:
        id2tok = cPickle.load(f)

In [16]:
embeddings = load_embeddings(config, tok2id)

INFO:Initialized embeddings.


## Training

In [25]:
from tf_birnn_classifier import TfBiRNNClassifier
from defs import PROJECT_DIR

In [26]:
model = TfBiRNNClassifier(embedding=embeddings, eta=0.001, use_attn = True,
                           ckpts_prefix='{}/checkpoints/birnn_model/birnn-model'.format(PROJECT_DIR),
                           summaries_dir='{}/summaries-birnn/'.format(PROJECT_DIR)
                          )
model.fit(tfConfig, restore_weights=False, batches_to_eval=250, max_iter=10)

INFO:Initializing data manager...
INFO:took 0 s
INFO:Building model...
INFO:took 0 s


fw outputs:  (?, 300, 50)
bw outputs:  (?, 300, 50)
all outputs:  (?, 300, 100)
all states:  (?, 100)
W_attn: (100, 25)


INFO:-- Initializing new model
INFO:Starting epoch 1
INFO: == Evaluating on development data after batch 250
INFO:Ran 40 batches
INFO:== Results for 40 batches of size 32
INFO:== Precision: 0.647, Recall: 0.026, F1: 0.049
INFO:-- Saving model


KeyboardInterrupt: 

## Evaluation

In [19]:
def print_example(pred, label, feats, length, email_id):
    ids_ = feats[:length][:, 0]
    print("==== {} ====".format(email_id))
    print("Predicted label: {}. True label: {}".format(LBLS[pred], LBLS[label]))
    print( " ".join((id2tok[i] for i in ids_)) )
    print("------------------------------------")

def print_examples(preds, outputs, inputs, lens, email_ids):
    for p, o, i, l, d in zip(preds, outputs, inputs, lens, email_ids):
        print_example(p, o, i, l, d)

In [21]:
outputs, preds, word_ids, email_ids = model.evaluate_tfdata(model.sess, dataset_name='dev', batch_lim=10, writer=None)

preds = np.asarray(preds); outputs = np.array(outputs)    
correct = preds == outputs
prec, rec, f1, _ = metrics.precision_recall_fscore_support(outputs, preds, average='binary')

print(" Predicted {} of {} examples correctly".format(np.sum(correct), correct.shape[0]) )
print(" Precision: {:1.3f}, Recall: {:1.3f}, F1: {:1.3f}".format(prec, rec, f1) )

INFO:Ran 10 batches


 Predicted 226 of 320 examples correctly
 Precision: 0.613, Recall: 0.496, F1: 0.548


In [25]:
preds, inputs, lens, outputs, email_ids, attns = model.predict()
print_examples(preds, outputs, inputs, lens, email_ids)

==== b'<26202349.1075849392470.JavaMail.evans@thyme>' ====
Predicted label: NORESPONSE. True label: NORESPONSE
<s> hey there ! how are you ? how 's your summer ? i 'm UUUNKKK the end of summer school and can not wait UUUNKKK it 's over . the class has been a new low point in terms of educational UUUNKKK UUUNKKK ! ... in fact , i 'm in class now . work is UUUNKKK UUUNKKK . UUUNKKK 'm thinking about taking advantage of sun 's flexible work deal and moving back up to the city . i do not like the south bay much . trying to get a happy hour together in august ... are you around ? let me know how you are ... UUUNKKK __________________________________________________ do you yahoo ! ? make international calls for as low as $ UUUNKKK with yahoo ! UUUNKKK http : UUUNKKK </s>
------------------------------------
==== b'<22146292.1075855935696.JavaMail.evans@thyme>' ====
Predicted label: REPLY. True label: REPLY
<s> services sally , this is the second of the two emails i am sending you . ? please 

In [ ]:
import matplotlib


In [ ]:
def make_attn_plot(word_ids, attn):
    